In [1]:
import tensorflow as tf
import keras
from keras import Model
from keras import layers
import numpy as np

2024-09-26 17:32:45.344870: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 17:32:45.410207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 17:32:45.437388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 17:32:45.445777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 17:32:45.490641: I tensorflow/core/platform/cpu_feature_guar


#------------------------------------------------------------------------------------#
##  THIS IS A TRAINING MODEL, IMPLEMENTION DETAIL MAY DIFFER FROM PRODUCTION MODEL  ##
#------------------------------------------------------------------------------------#


# TODO
# THIS IS AN ABSTRACT BACKBONE SCRIPT, ADD ACTUAL IMPLEMENTATION LATER

# NOTE
# model detail

In [ ]:
# global variables
model = None # public

In [2]:
# custom layer definition
class Conv2Plus1D(layers.Layer):
    def __init__(self, kernel_size, filters = 1, strides = (1,1,1), padding = 'valid'):
        """kernel_size is depth width height"""
        super().__init__()
        wh_stride = (1, strides[1], strides[2])
        t_stride = (strides[0], 1, 1)
        self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]), strides = wh_stride,
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1), strides = t_stride,
                      padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

In [3]:
# hand model
class HandModel(Model):
    # run it every 2 frames in order to give it 2 temporal stride
    def __init__(self):
        super().__init__()
        self.conv21 = Conv2Plus1D(kernel_size = (25,3,3))
        self.ln = layers.LayerNormalization()

    def call(self, x, training= False):
        x = self.conv21(x)
        return self.ln(x, training= training)


In [ ]:
# pose model
class PoseModel(Model):
    def __init__(self, kernel_size = (13,3)):
        super().__init__()
        self.kernel_size = kernel_size
        self.dense = layers.Dense(3, activation='relu')
        self.conv2 = layers.Conv2D(filters=1, kernel_size = kernel_size)
        self.ln = layers.LayerNormalization()

    def call(self, input, training= False):
        # input: outer layer < batch time channels features > inner layer
        temp1 = []
        temp2 = []
        print(input.shape)
        for batch in input:
            for frame in batch:
                for channel in frame:
                    # combine each channel into 3 scalar values using FC 
                    x = tf.expand_dims(channel, axis=0)
                    x = self.dense(x)
                    print('x:',x)
                    # reconstruct matrix
                    temp1.append(tf.squeeze(x))
                x = tf.stack(temp1)
                x = tf.transpose(x)
                temp2.append(x)
                temp1.clear()
            temp1.clear()
            temp1.append(tf.stack(temp2))
        pre_conv = tf.stack(temp1)
        x = self.conv2(pre_conv)
        return self.ln(x, training= training)
        

In [ ]:
# the main model class
class SLRModel(Model):
    def __init__(self):
        super().__init__()
        pass

    def call(self, inputs):
        # inputs 0: L, 1: R, 2: Pose
        
        pass


# the part that doesnt get executed when imported
if __name__ == "__main__":
    pass


def get_model():
    return model

def set_model(new_model):
    global model
    model = new_model

def train_or_fit():
    pass